In [2]:
from sqlalchemy import create_engine

## Create engine

In [3]:
engine = create_engine('sqlite:///college.db', echo = True)

In [42]:
# sqlite in-memory
#engine = create_engine('sqlite://', echo = True)

In [43]:
engine

Engine(sqlite://)

## MetaData for table creation

In [44]:
from sqlalchemy import MetaData
# Metadata contains definitions of tables and associated objects such as index, view, triggers, etc.
meta = MetaData()

In [45]:
# you can use the metadata obj to hold tables info
from sqlalchemy import Table, Column, Integer, String


In [46]:
# create a table object called students
students = Table(
    "students",
    meta, # the metadata obj to hold this table info
    Column('id', Integer, primary_key = True), 
    Column('name', String), 
    Column('lastname', String), 
)

In [47]:
# now you can actually use the metadata obj to execute
# create tables. It takes the engine as argument
# now this actually creates the database
meta.create_all(engine)

2021-04-21 20:36:53,165 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-21 20:36:53,167 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 20:36:53,168 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-21 20:36:53,168 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 20:36:53,169 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("students")
2021-04-21 20:36:53,170 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 20:36:53,172 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2021-04-21 20:36:53,172 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 20:36:53,173 INFO sqlalchemy.engine.base.Engine COMMIT


## Use table methods to create expression

In [48]:
# create a sql query expression
ins = students.insert()
str(ins)

'INSERT INTO students (id, name, lastname) VALUES (:id, :name, :lastname)'

In [49]:
# bind and compile with values inserted
ins = ins.values(id=1, name="Bob", lastname="Smith")

In [50]:
# see the params
ins.compile().params

{'id': 1, 'name': 'Bob', 'lastname': 'Smith'}

In [51]:
str(ins)  # here you don't see it though

'INSERT INTO students (id, name, lastname) VALUES (:id, :name, :lastname)'

## Execute expression

In [52]:
# create connection
conn = engine.connect()

In [53]:
# we can use the expression or string query to execute
conn.execute(ins)

2021-04-21 20:36:59,311 INFO sqlalchemy.engine.base.Engine INSERT INTO students (id, name, lastname) VALUES (?, ?, ?)
2021-04-21 20:36:59,312 INFO sqlalchemy.engine.base.Engine (1, 'Bob', 'Smith')
2021-04-21 20:36:59,314 INFO sqlalchemy.engine.base.Engine COMMIT


In [54]:
# another 1
ins = students.insert().values(name = 'Ravi', lastname = 'Kapoor')
result = conn.execute(ins)


2021-04-21 20:36:59,969 INFO sqlalchemy.engine.base.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-04-21 20:36:59,970 INFO sqlalchemy.engine.base.Engine ('Ravi', 'Kapoor')
2021-04-21 20:36:59,972 INFO sqlalchemy.engine.base.Engine COMMIT


In [55]:
result = conn.execute("select * from students")

2021-04-21 20:37:01,187 INFO sqlalchemy.engine.base.Engine select * from students
2021-04-21 20:37:01,188 INFO sqlalchemy.engine.base.Engine ()


In [56]:
result.fetchall()

[(1, 'Bob', 'Smith'), (2, 'Ravi', 'Kapoor')]

In [ ]:
ins = students.insert().values(name = 'Happy', lastname = 'Face')
result = conn.execute(ins)

In [ ]:
result.fetchall()

## Textual SQL

In [57]:
from sqlalchemy import text
t = text("SELECT * FROM students")
result = conn.execute(t)

2021-04-21 20:37:03,906 INFO sqlalchemy.engine.base.Engine SELECT * FROM students
2021-04-21 20:37:03,908 INFO sqlalchemy.engine.base.Engine ()


In [58]:
result.fetchall()

[(1, 'Bob', 'Smith'), (2, 'Ravi', 'Kapoor')]

In [59]:
# You can format the query using text
s = text("select students.name, students.lastname from students where students.name between :x and :y")
conn.execute(s, x = 'A', y = 'L').fetchall()

2021-04-21 20:37:06,254 INFO sqlalchemy.engine.base.Engine select students.name, students.lastname from students where students.name between ? and ?
2021-04-21 20:37:06,255 INFO sqlalchemy.engine.base.Engine ('A', 'L')


[('Bob', 'Smith')]

In [60]:
from sqlalchemy.sql import select
s = select([text("students.name, students.lastname from students")]).where(text("students.name between :x and :y"))
conn.execute(s, x = 'A', y = 'L').fetchall()

2021-04-21 20:37:06,626 INFO sqlalchemy.engine.base.Engine SELECT students.name, students.lastname from students 
WHERE students.name between ? and ?
2021-04-21 20:37:06,627 INFO sqlalchemy.engine.base.Engine ('A', 'L')


[('Bob', 'Smith')]

In [61]:
conn.close()